In [50]:
import os
import random

In [51]:
class gate_dlinkedList:
    def __init__(self, gate_id):
        self.gate_id = gate_id
        self.bucket_side = None
        self.lock = False
        self.next = None
        self.prev = None
        self.gain = None
        
    def set_next(self, next_gate):
        self.next = next_gate
        if next_gate:
            next_gate.prev = self
        return self.next
    
    def move_gate(self, new_gain):

        self.remove_gate()

        if new_gain in self.bucket_side.keys():
            self.bucket_side[new_gain].set_next(self)
        elif new_gain not in self.bucket_side.keys():
            self.bucket_side[new_gain] = self
        self.gain = new_gain
        

    def remove_gate(self):
        if self.gain in self.bucket_side.keys() and not self.next:
            if self.prev:
                self.bucket_side[self.gain] = self.prev
            else: #not self.prev
                del self.bucket_side[self.gain]
        
        if self.prev:
            self.prev.next = self.next
        if self.next:
            self.next.prev = self.prev
        self.next = None
        self.prev = None
        self.gain = None

    def __repr__(self):
        if self.next:
            next_id = self.next.gate_id  
        else:
            next_id = 'None'
        prev_id = self.prev.gate_id if self.prev else 'None'
        return f"Gate(Id:{self.gate_id},next:{next_id},prev:{prev_id},lock:{self.lock})"

In [52]:
gate1 = gate_dlinkedList(1)
gate2 = gate_dlinkedList(2)
gate3 = gate_dlinkedList(3)
# print(str(gate1))
print(str(gate1))
curr_gate = gate1.set_next(gate2)

print(str(curr_gate))
last_gate = gate2.set_next(gate3)
# print(str(gate1))
print(str(last_gate))

print(gate3)

Gate ID: 1
Lock Status: False

Gate ID: 2
Lock Status: False
Prev Gate: 1

Gate ID: 3
Lock Status: False
Prev Gate: 2

Gate ID: 3
Lock Status: False
Prev Gate: 2



In [53]:
class readFile:
    def __init__(self):
        self.circuit_data = None
        
    def read_circuit(self, circuit_folder_path):
        netD_file = None
        are_file = None
        for file in os.listdir(circuit_folder_path):
            if file.endswith('.netD'):
                netD_file = os.path.join(circuit_folder_path, file)
            if file.endswith('.are'):
                are_file = os.path.join(circuit_folder_path, file)
        if netD_file and are_file:
            self.circuit_data = self.parse_net_files(netD_file, are_file)
            return self.circuit_data
        else:
            print("Error Message: Missing Files")
            return None

    def parse_net_files(self, netD_file, are_file):
        # network information
        nets, connections, nodes, modules, pad_offset = self.read_netD_file(netD_file)
        # module areas
        areas = self.read_are_file(are_file)
        # network information with pin directions

        self.circuit_data = {
            'areas': areas,
            'netD_nets': nets,
            'connections': int(connections),
            'nodes': int(nodes),
            'modules': int(modules),
            'pad_offset': int(pad_offset)
        }
        return self.circuit_data
    
    def read_are_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        module_areas = {}
        for line in lines:
            module_id, area = line.split()
            module_areas[module_id] = int(area)
        return module_areas

    def read_netD_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        self.circuit_info = tuple(item.replace('\n', '') for item in lines[0:5])
        _, connections, nodes, modules, pad_offset = self.circuit_info
        lines = lines[5:]
        nets = {}
        current_key = None
        for line in lines:
            tokens = line.split()
            if tokens[1] == 's':
                current_key = tokens[0]
                nets[current_key] = []
            elif tokens[1] == 'l':
            ## update from here
                if current_key is not None:  # Check if a valid key is assigned
                    nets[current_key].append(tokens[0])
        # Ensure all keys have an entry in the dictionary
        for token in lines:
            if token.startswith('p') and token.split()[1] != 's':
                nets[token.split()[0]] = []
        return nets, connections, nodes, modules, pad_offset

In [54]:
class FM_algo:
    def __init__(self, circuit_folder_path):
        self.circuit_data = readFile().read_circuit(circuit_folder_path)
        self.circuit_connections = self.circuit_data['netD_nets']
        print(self.circuit_connections)
        self.bucket_gains1 = dict()
        self.bucket_gains2 = dict()
        self.vertex = dict()
        test_best_cutsize = self.fd_algo()
        print('Fiduccia-Mattheyses Algorithm Complete!')
        print(f'Best Cut Size: {test_best_cutsize}')
        
    def fd_algo(self):
        #partition circuit
        self.partition1, self.partition2 = self.partition_circuit()
        #initialize buckets
        self.bucket_gains1 = self.initialize_buckets(self.partition1, self.bucket_gains1, self.bucket_gains1)
        self.bucket_gains2 = self.initialize_buckets(self.partition2, self.bucket_gains2, self.bucket_gains2)
        cutsize = self.calculate_cutsize()
        
        ##TODO: Begin Fiduccia-Mattheyses Algorithm below
        while(True):
            last_cutsize = cutsize
            print('initial cutsize: ', cutsize)
            cutsize = self.fm_pass()
            
            if cutsize > last_cutsize:
                break
            self.reinitialize_buckets()
        return last_cutsize

    def fm_pass(self):
        best_cutsize = float('inf')
        best_bucket1 = self.bucket_gains1
        best_bucket2 = self.bucket_gains2
        locked_gates_count = 0
        
        while (not self.check_gate_locks(locked_gates_count)): #Stop if all gates are locked
            max_gain_node, _ = self.calculate_max_gain()
            #move to opposite side of bucket && remove gate from bucket & lock gate
            max_gain_node.remove_gate()
            max_gain_node.lock = True
            locked_gates_count += 1
            
            if max_gain_node.bucket_side == self.bucket_gains1:
                max_gain_node.bucket_side = self.bucket_gains2
            elif max_gain_node.bucket_side == self.bucket_gains2:
                max_gain_node.bucket_side = self.bucket_gains1

            #iterate through connected nodes to update gain buckets
            for net in self.circuit_connections[max_gain_node.gate_id]:
                net_node = self.vertex[net]
                net_side = net_node.bucket_side
                #TODO:re-calculate gain
                new_gain = self.calculate_gain(net)
                if net_side == self.bucket_gains1:
                    net_node.move_gate(new_gain)
                else: #if net_side == self.bucket_gains2:
                    net_node.move_gate(new_gain)
                net_node.gain = new_gain
                
                current_cutsize = self.calculate_cutsize()
                print("New Cutsize: ", current_cutsize)
                #TODO: Rollback to best observed cutsize
                if current_cutsize < best_cutsize:
                    best_cutsize = current_cutsize
                    best_bucket1 = self.bucket_gains1
                    best_bucket2 = self.bucket_gains2
        self.bucket_gains1 = best_bucket1
        self.bucket_gains2 = best_bucket2
        new_cutsize = best_cutsize
        return new_cutsize
    
    def reinitialize_buckets(self):
        for gate, node in self.vertex.items():
            #unlock gate
            node.lock = False
            gain = self.calculate_gain(gate)
            node.gain = gain

            if gain in node.bucket_side.keys():
                node.bucket_side[gain].set_next(node)    
            else:
                node.bucket_side[gain] = node

    def check_gate_locks(self, locked_gates_count):
        # return True if all gates are locked
        return locked_gates_count == len(self.vertex) 

    def calculate_cutsize(self):
        cut_size = 0
        for gate, connected_gates in self.circuit_connections.items():
            current_gate_node = self.vertex[gate]
            current_gate_side = current_gate_node.bucket_side
            for connected_gate in connected_gates:
                connected_gate_node = self.vertex[connected_gate]
                connected_gate_side = connected_gate_node.bucket_side
                if current_gate_side != connected_gate_side:
                    cut_size += 1
                    break 
        return cut_size

    def calculate_max_gain(self):
        if self.bucket_gains1: max_gain1 = max(self.bucket_gains1.keys())
        else: max_gain1 = float('-inf')

        if self.bucket_gains2: max_gain2 = max(self.bucket_gains2.keys()) 
        else: max_gain2 = float('-inf')

        if max_gain1 >= max_gain2: 
            selected_bucket = self.bucket_gains1  
        else: 
            selected_bucket = self.bucket_gains2

        overall_max_gain = max(max_gain1, max_gain2)

        if overall_max_gain != float('-inf'): max_gain_node = selected_bucket[overall_max_gain]  
        else: max_gain_node = None

        return max_gain_node, overall_max_gain
    
    def partition_circuit(self):
        circuit_nodes = list(self.circuit_connections.keys())
        random.shuffle(circuit_nodes)
        partition = len(circuit_nodes) // 2

        partition1 = circuit_nodes[:partition]
        partition2 = circuit_nodes[partition:] 
        return partition1, partition2
    
    def calculate_gain(self, gate):
        external_sum = 0
        internal_sum = 0
        
        if self.vertex[gate].bucket_side == 1:
            current_partition = self.partition1
        else: #if gain side == 2
            current_partition = self.partition2

        for next_gate in self.circuit_connections[gate]:
            if next_gate in current_partition: internal_sum += 1
            else:  external_sum += 1
        gain = external_sum - internal_sum
        return gain
        
    def initialize_buckets(self, partition, bucket, side):
        for gate in partition:
            current_gate = gate_dlinkedList(gate)
            current_gate.bucket_side = side
            self.vertex[gate] = current_gate
            gain = self.calculate_gain(gate)
            current_gate.gain = gain
            if gain not in bucket.keys():
                bucket[gain] = current_gate #gate must be doubly linked list OOC
            else:
                bucket[gain] = bucket[gain].set_next(current_gate)
            #add gate node to vertex bucket for lookup

        return bucket

In [ ]:
def main():
    circuit_folder_path = "test_simple"
    fm = FM_algo(circuit_folder_path)
if __name__ == "__main__":
    main()

In [63]:
circuit_folder_path = "test_simple"
fm = FM_algo(circuit_folder_path)

print('left partition gains', fm.bucket_gains1)
print('right partition gains', fm.bucket_gains2)
print('vertex', fm.vertex)

left partition gains {-1: <__main__.gate_dlinkedList object at 0x00000257AE86B400>, 0: <__main__.gate_dlinkedList object at 0x00000257AE86B940>, 1: <__main__.gate_dlinkedList object at 0x00000257AE86BFA0>}
right partition gains {2: <__main__.gate_dlinkedList object at 0x00000257AE86BD00>, -2: <__main__.gate_dlinkedList object at 0x00000257AE86BC10>, 0: <__main__.gate_dlinkedList object at 0x00000257AE86BB20>}
vertex {'a2': <__main__.gate_dlinkedList object at 0x00000257AE86B400>, 'p2': <__main__.gate_dlinkedList object at 0x00000257AE86B940>, 'a3': <__main__.gate_dlinkedList object at 0x00000257AE86BFA0>, 'a0': <__main__.gate_dlinkedList object at 0x00000257AE86B6D0>, 'a1': <__main__.gate_dlinkedList object at 0x00000257AE86BD00>, 'p1': <__main__.gate_dlinkedList object at 0x00000257AE86BC10>, 'p3': <__main__.gate_dlinkedList object at 0x00000257AE86BB20>}


In [56]:
#### TEST FUNCTION ####
#### inputs from https://vlsicad.ucsd.edu/UCLAWeb/cheese/ispd98.html#Benchmark%20File%20Format ####
#### example with 4 cells, 3 pads, 5 nets and 13 pins ####
circuit_folder_path = "test_simple"
fm = FM_algo(circuit_folder_path)
print(fm.circuit_data['netD_nets'])


{'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3'], 'p2': [], 'p3': []}


In [57]:
fm.circuit_data['netD_nets']

{'p1': ['a0', 'a1'],
 'a0': ['a2', 'a3'],
 'a1': ['a2', 'a3'],
 'a2': ['p2'],
 'a3': ['p3'],
 'p2': [],
 'p3': []}

In [58]:
fm.circuit_data['areas']

{'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}

In [59]:
def main():
    pass

if __name__ == "__main__":
    main()

In [60]:
print(fm.circuit_data['areas'])
print(fm.circuit_data['netD_nets'])


{'a0': 1, 'a1': 3, 'a2': 4, 'a3': 2, 'p1': 0, 'p2': 0, 'p3': 0}
{'p1': ['a0', 'a1'], 'a0': ['a2', 'a3'], 'a1': ['a2', 'a3'], 'a2': ['p2'], 'a3': ['p3'], 'p2': [], 'p3': []}
